# SIFT

SIFT is a feature based image registration algorithm developed by David Lowe in 1999. It works by detecting feature points between two images and switching them based on common points between the two images. Its applications range from basic panorama creation to satellite imagery, and facial recognition in video.

There are five major steps:

- **scale-space construction**: convolving an image repeatedly using Gaussian filter with changing scales
- **scale space extrema detection**: Difference of Gaussian (DoG) images are computed from Gaussian-blurred images
- **keybpoint localization**: identified as local extrma of DoG across the scales.
- **orientation assignment**: low contrast keypoints and edge response points are discarded
- **defining keypoint descriptors**: assigned one or more orientations based on the local image gradient directions

Orientation histograms are formed over heighborhoods of keypoints. A normalized 128-dimensional vector is computed for each keypoint as its _descriptor_. Nearest neighbor of a keypoint in the first image is identified from a database of keypoints from the second image. RANSAC is used to remove outpiers and to compute the transformation parameters between a pari of frames. Finally, they are warped using transformation parameters to create the mosaic image.

## But we're not using SIFT here, we're using AKAZE
[OpenCV python matcher](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_matcher/py_matcher.html).

In [183]:
import cv2
import imageio

In [184]:
def get_akaze_keypoints(im):
    akaze = cv2.AKAZE_create()
    return akaze.detectAndCompute(im, None)

In [185]:
im1 = imageio.imread('../data/T1_segment_1.tif')
im2 = imageio.imread('../data/T1_segment_2.tif')

(kp_1, desc_1) = get_akaze_keypoints(im1)
(kp_2, desc_2) = get_akaze_keypoints(im2)

In [186]:
bf = cv2.BFMatcher()
# Match descriptors.
matches = bf.knnMatch(desc_1, desc_2, k=2)

# Sort them in the order of their distance.
# matches = sorted(matches, key = lambda x:x.distance)

In [187]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline

def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side
    This could use PIL.Image.paste, but it complains about tuples.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha, hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width))
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img

In [ ]:
im3 = concat_images(im1, im2)

fig=plt.figure(figsize=(18, 16), dpi=72)
plt.imshow(im3)

ax = plt.gca()
for m,n in matches[::50]:
    if m.distance < 0.75*n.distance:
        kp_a = kp_1[m.queryIdx].pt
        kp_b = kp_2[m.trainIdx].pt
        l = mlines.Line2D([kp_a[0], kp_b[0]+im1.shape[0]], 
                          [kp_a[1], kp_b[1]], color='r')
        ax.add_line(l)
        
plt.show()